In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/최종데이터/final_data.csv')

In [5]:
df['진행상황'].value_counts()

df['진행상황'] = df['진행상황'].apply(lambda x: 0 if x == 'uploaded' else 1)

In [6]:
df['진행상황'].value_counts()

진행상황
0    149
1     98
Name: count, dtype: int64

진행상황이 0이면 uploaded이고 그 외는 전부 1이다.

In [11]:
df.to_csv('D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/최종데이터/final_data.csv', index=False)

In [12]:
df

,Username,follower,following,City,Country,median_views,median_comments,average_views,average_comments,last_video_date,verified,top_hashtags,진행상황
0,theonlyhaven,1200000,77,Singapore,SG,17.6K,8.0,134.6K,32.0,2019-12-19,False,"rlholiday, poloralphlauren, rlholidaytree, ral...",1
1,turquoise.dec,755000,122,Singapore,SG,117.8K,34.0,341.6K,153.0,2021-02-14,False,"yslbeauty, yslbeautysg, yslmakeup, ysltheinks",1
2,ttalgi.ellie,1373,726,Sarzana,Italy,932,5.0,4.3K,6.0,2024-02-07,False,"fyp, support, unbox, outfits, outfitinspo, hau...",0
3,srsaskin_,1439,890,Birmingham,GB,1.1K,19.0,2.7K,28.0,2024-02-03,False,"kbeauty, koreanskincare, teca, fyp, abib, abib...",0
4,_anjello,949,681,Concord,US,389,24.0,529,27.0,2024-01-14,False,"skincare, capcut, kbeauty, koreanskincare, ski...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,aglimpseofglow,60,82,La Habra,US,948,0.0,952,1.0,2022-12-26,False,"ugccontentcreators, skincareinfluencer, makeup...",0
243,mariana.ugc,2313,1075,Green Bay,US,461,8.0,3.3K,11.0,2023-04-15,False,"gorchicunboxing, gorchichaul, gorchic, fallhau...",0
244,mariana.ugc,2313,1075,Green Bay,US,461,8.0,3.3K,11.0,2023-04-15,False,"gorchicunboxing, gorchichaul, gorchic, fallhau...",0
245,tashfia____,1425,360,Dhaka,Bangladesh,1.3K,3.0,3.4K,7.0,2023-11-09,False,"girly, beautytok, beauty, girlythings, makeup,...",1


In [13]:
import re
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api

In [14]:
# 1. 모델 로드 (처음에만 오래 걸림)
print("모델 로드 중...")
model = api.load("fasttext-wiki-news-subwords-300")  # 영어 fastText

모델 로드 중...
[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
# # 2. 내 홍보 키워드 정의 (사용자 입력)
# input_str = input("홍보 키워드를 쉼표(,)로 구분해서 입력하세요 (예: k-beauty,cosmetics,skincare): ")
# my_keywords = [kw.strip() for kw in input_str.split(',') if kw.strip()]
# print("입력된 키워드:", my_keywords)

# keyword_vecs = [model[w] for w in my_keywords if w in model]
# mean_keyword_vec = np.mean(keyword_vecs, axis=0).reshape(1, -1)# 2. 내 홍보 키워드 정의 (사용자 입력)
# input_str = input("홍보 키워드를 쉼표(,)로 구분해서 입력하세요 (예: k-beauty,cosmetics,skincare): ")
# my_keywords = [kw.strip() for kw in input_str.split(',') if kw.strip()]
# print("입력된 키워드:", my_keywords)

# keyword_vecs = [model[w] for w in my_keywords if w in model]
# mean_keyword_vec = np.mean(keyword_vecs, axis=0).reshape(1, -1)

In [15]:
# 2. 내 홍보 키워드 정의
my_keywords = ["k-beauty", "cosmetics", "skincare"]  # 원하는 키워드로 변경
keyword_vecs = [model[w] for w in my_keywords if w in model]
mean_keyword_vec = np.mean(keyword_vecs, axis=0).reshape(1, -1)

# 3. 각 row의 top_hashtags와 내 키워드의 의미적 유사도 계산
def calc_hashtag_similarity(hashtags):
    if pd.isna(hashtags):
        return np.nan
    tags = re.split(r'[\s,]+', hashtags.strip().lower())
    tag_vecs = [model[tag] for tag in tags if tag in model]
    if not tag_vecs:
        return np.nan
    mean_tag_vec = np.mean(tag_vecs, axis=0).reshape(1, -1)
    return cosine_similarity(mean_keyword_vec, mean_tag_vec)[0][0]

df['hashtag_similarity'] = df['top_hashtags'].apply(calc_hashtag_similarity)

In [16]:
df

,Username,follower,following,City,Country,median_views,median_comments,average_views,average_comments,last_video_date,verified,top_hashtags,진행상황,hashtag_similarity
0,theonlyhaven,1200000,77,Singapore,SG,17.6K,8.0,134.6K,32.0,2019-12-19,False,"rlholiday, poloralphlauren, rlholidaytree, ral...",1,NaN
1,turquoise.dec,755000,122,Singapore,SG,117.8K,34.0,341.6K,153.0,2021-02-14,False,"yslbeauty, yslbeautysg, yslmakeup, ysltheinks",1,NaN
2,ttalgi.ellie,1373,726,Sarzana,Italy,932,5.0,4.3K,6.0,2024-02-07,False,"fyp, support, unbox, outfits, outfitinspo, hau...",0,0.512804
3,srsaskin_,1439,890,Birmingham,GB,1.1K,19.0,2.7K,28.0,2024-02-03,False,"kbeauty, koreanskincare, teca, fyp, abib, abib...",0,0.459083
4,_anjello,949,681,Concord,US,389,24.0,529,27.0,2024-01-14,False,"skincare, capcut, kbeauty, koreanskincare, ski...",0,0.682939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,aglimpseofglow,60,82,La Habra,US,948,0.0,952,1.0,2022-12-26,False,"ugccontentcreators, skincareinfluencer, makeup...",0,0.365554
243,mariana.ugc,2313,1075,Green Bay,US,461,8.0,3.3K,11.0,2023-04-15,False,"gorchicunboxing, gorchichaul, gorchic, fallhau...",0,0.714643
244,mariana.ugc,2313,1075,Green Bay,US,461,8.0,3.3K,11.0,2023-04-15,False,"gorchicunboxing, gorchichaul, gorchic, fallhau...",0,0.714643
245,tashfia____,1425,360,Dhaka,Bangladesh,1.3K,3.0,3.4K,7.0,2023-11-09,False,"girly, beautytok, beauty, girlythings, makeup,...",1,0.663123


In [ ]:
df['hashtag_similarity'] = df['hashtag_similarity'].fillna(0)

In [18]:
from scipy.stats import ttest_ind

group0 = df[df['진행상황'] == 0]['hashtag_similarity']
group1 = df[df['진행상황'] == 1]['hashtag_similarity']

t_stat, p_val = ttest_ind(group0, group1, nan_policy='omit')
print(f"t-통계량: {t_stat:.3f}, p-value: {p_val:.3f}")

t-통계량: 0.964, p-value: 0.336


In [19]:
import re

def to_number(x):
    if pd.isna(x):
        return np.nan
    x = str(x).replace(',', '').strip().lower()
    # 'k', 'm' 단위 처리
    if x.endswith('k'):
        return float(x[:-1]) * 1000
    if x.endswith('m'):
        return float(x[:-1]) * 1000000
    # 일반 숫자
    try:
        return float(x)
    except:
        return np.nan

df['median_views'] = df['median_views'].apply(to_number)
df['median_comments'] = df['median_comments'].apply(to_number)
df['follower'] = df['follower'].apply(to_number)

df['median_views_per_follower'] = df['median_views'] / df['follower']
df['median_comments_per_follower'] = df['median_comments'] / df['follower']

In [21]:
df

,Username,follower,following,City,Country,median_views,median_comments,average_views,average_comments,last_video_date,verified,top_hashtags,진행상황,hashtag_similarity,median_views_per_follower,median_comments_per_follower
0,theonlyhaven,1200000.0,77,Singapore,SG,17600.0,8.0,134.6K,32.0,2019-12-19,False,"rlholiday, poloralphlauren, rlholidaytree, ral...",1,0.000000,0.014667,0.000007
1,turquoise.dec,755000.0,122,Singapore,SG,117800.0,34.0,341.6K,153.0,2021-02-14,False,"yslbeauty, yslbeautysg, yslmakeup, ysltheinks",1,0.000000,0.156026,0.000045
2,ttalgi.ellie,1373.0,726,Sarzana,Italy,932.0,5.0,4.3K,6.0,2024-02-07,False,"fyp, support, unbox, outfits, outfitinspo, hau...",0,0.512804,0.678806,0.003642
3,srsaskin_,1439.0,890,Birmingham,GB,1100.0,19.0,2.7K,28.0,2024-02-03,False,"kbeauty, koreanskincare, teca, fyp, abib, abib...",0,0.459083,0.764420,0.013204
4,_anjello,949.0,681,Concord,US,389.0,24.0,529,27.0,2024-01-14,False,"skincare, capcut, kbeauty, koreanskincare, ski...",0,0.682939,0.409905,0.025290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,aglimpseofglow,60.0,82,La Habra,US,948.0,0.0,952,1.0,2022-12-26,False,"ugccontentcreators, skincareinfluencer, makeup...",0,0.365554,15.800000,0.000000
243,mariana.ugc,2313.0,1075,Green Bay,US,461.0,8.0,3.3K,11.0,2023-04-15,False,"gorchicunboxing, gorchichaul, gorchic, fallhau...",0,0.714643,0.199308,0.003459
244,mariana.ugc,2313.0,1075,Green Bay,US,461.0,8.0,3.3K,11.0,2023-04-15,False,"gorchicunboxing, gorchichaul, gorchic, fallhau...",0,0.714643,0.199308,0.003459
245,tashfia____,1425.0,360,Dhaka,Bangladesh,1300.0,3.0,3.4K,7.0,2023-11-09,False,"girly, beautytok, beauty, girlythings, makeup,...",1,0.663123,0.912281,0.002105


In [22]:
from scipy.stats import ttest_ind

# median_views_per_follower에 대한 t-검정
group0_views = df[df['진행상황'] == 0]['median_views_per_follower']
group1_views = df[df['진행상황'] == 1]['median_views_per_follower']
t_stat_views, p_val_views = ttest_ind(group0_views, group1_views, nan_policy='omit')
print(f"[median_views_per_follower] t-통계량: {t_stat_views:.3f}, p-value: {p_val_views:.3f}")

# median_comments_per_follower에 대한 t-검정
group0_comments = df[df['진행상황'] == 0]['median_comments_per_follower']
group1_comments = df[df['진행상황'] == 1]['median_comments_per_follower']
t_stat_comments, p_val_comments = ttest_ind(group0_comments, group1_comments, nan_policy='omit')
print(f"[median_comments_per_follower] t-통계량: {t_stat_comments:.3f}, p-value: {p_val_comments:.3f}")

[median_views_per_follower] t-통계량: -0.602, p-value: 0.548
[median_comments_per_follower] t-통계량: 3.471, p-value: 0.001
